In [9]:
import pandas as pd
import numpy as np
from ast import literal_eval
from nltk.stem.snowball import SnowballStemmer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [20]:
md = pd.read_csv('/Users/ayushsingh/Desktop/Movie-Reco/data/movies_metadata.csv')

/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [21]:
md.shape

(45466, 24)

In [22]:
credits = pd.read_csv('/Users/ayushsingh/Desktop/Movie-Reco/data/credits.csv')
keywords = pd.read_csv('/Users/ayushsingh/Desktop/Movie-Reco/data/keywords.csv')

In [23]:
md['genres'] = md['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
md = md.drop([19730, 29503, 35587])
md['year'] = pd.to_datetime(md['release_date'], errors='coerce').apply(lambda x: str(x).split('-')[0] if x != np.nan else np.nan)

In [24]:
md.shape

(45463, 25)

In [26]:
keywords['id'] = keywords['id'].astype('int')
credits['id'] = credits['id'].astype('int')
md['id'] = md['id'].astype('int')

In [27]:
md = md.merge(credits, on='id')
md = md.merge(keywords, on='id')

In [38]:
md['cast'] = md['cast'].apply(literal_eval)

In [39]:
md['crew'] = md['crew'].apply(literal_eval)

In [40]:
md['keywords'] = md['keywords'].apply(literal_eval)

In [41]:
md['cast_size'] = md['cast'].apply(lambda x: len(x))

In [42]:
md['crew_size'] = md['crew'].apply(lambda x: len(x))

In [43]:
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

In [44]:
md['director'] = md['crew'].apply(get_director)

In [45]:
md['cast'] = md['cast'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
md['cast'] = md['cast'].apply(lambda x: x[:3] if len(x) >=3 else x)

In [46]:
md['keywords'] = md['keywords'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [47]:
md['cast'] = md['cast'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [48]:
md['director'] = md['director'].astype('str').apply(lambda x: str.lower(x.replace(" ", "")))
md['director'] = md['director'].apply(lambda x: [x,x, x])

In [56]:
s = md.apply(lambda x: pd.Series(x['keywords']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'keyword'

In [57]:
s = s.value_counts()
s[:10]

woman director      3128
independent film    1942
murder              1314
based on novel       841
musical              734
sex                  685
violence             651
biography            641
nudity               636
revenge              626
Name: keyword, dtype: int64

In [58]:
s = s[s > 1]

In [59]:
stemmer = SnowballStemmer('english')
stemmer.stem('dogs')

'dog'

In [60]:
def filter_keywords(x):
    words = []
    for i in x:
        if i in s:
            words.append(i)
    return words

In [62]:
md['keywords'] = md['keywords'].apply(filter_keywords)
md['keywords'] = md['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])
md['keywords'] = md['keywords'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [65]:
md['keywords']

0        [jealousi, toy, boy, friendship, friend, rival...
1        [boardgam, disappear, basedonchildren'sbook, n...
2           [fish, bestfriend, duringcreditssting, oldmen]
3        [basedonnovel, interracialrelationship, single...
4        [babi, midlifecrisi, confid, age, daughter, mo...
5        [robberi, detect, bank, obsess, chase, shoot, ...
6        [pari, brotherbrotherrelationship, chauffeur, ...
7                                                       []
8                  [terrorist, hostag, explos, vicepresid]
9        [cuba, falselyaccus, secretident, computerviru...
10       [whitehous, usapresid, newlov, widow, wildlife...
11                                        [dracula, spoof]
12            [wolf, alaska, dog, goos, bearattack, dogsl]
13       [usapresid, presidentialelect, watergatescand,...
14        [exoticisland, treasur, map, ship, scalp, pirat]
15       [poker, drugabus, 1970s, overdos, illegalprost...
16       [bowl, basedonnovel, servant, countrylif, jane.

In [66]:
md['soup'] = md['keywords'] + md['cast'] + md['director'] + md['genres']
md['soup'] = md['soup'].apply(lambda x: ' '.join(x))

In [67]:
count = CountVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
count_matrix = count.fit_transform(md['soup'])

In [68]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [70]:
md = md.reset_index()
titles = md['title']
indices = pd.Series(md.index, index=md['title'])

In [72]:
def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

In [77]:
get_recommendations('The Dark Knight Rises').head(10)

12589    The Dark Knight
10210      Batman Begins
2486           Following
26110          Doodlebug
26111          Doodlebug
11463       The Prestige
45843            Dunkirk
15651          Inception
5302            Insomnia
4126             Memento
Name: title, dtype: object

In [89]:
vote_counts = md[md['vote_count'].notnull()]['vote_count'].astype('int')
vote_averages = md[md['vote_average'].notnull()]['vote_average'].astype('int')
C = vote_averages.mean()
C

5.238696808510638

In [90]:
def improved_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]
    
    movies = md.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'year']]
    vote_counts = movies[movies['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = movies[movies['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(0.60)
    qualified = movies[(movies['vote_count'] >= m) & (movies['vote_count'].notnull()) & (movies['vote_average'].notnull())]
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('int')
    qualified['wr'] = qualified.apply(weighted_rating, axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(10)
    return qualified

def weighted_rating(x):
    m = vote_counts.quantile(0.60)
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+m) * R) + (m/(m+v) * C)

In [91]:
improved_recommendations('Inception')

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-

,title,vote_count,vote_average,year,wr
12589,The Dark Knight,12269,8,2008,7.996628
23076,Interstellar,11187,8,2014,7.996302
11463,The Prestige,4510,8,2006,7.990847
4126,Memento,4168,8,2000,7.990098
18442,The Dark Knight Rises,9263,7,2012,6.997152
10210,Batman Begins,7511,7,2005,6.996490
45843,Dunkirk,2712,7,2017,6.990312
5302,Insomnia,1181,6,2002,5.990452
8221,Sky Captain and the World of Tomorrow,445,5,2004,5.007784
21897,Don Jon,1708,5,2013,5.002078
